#Visualizing your molecule in property space using MolAtlas!

- Please prepare the properties of the target molecules at the B3LYP/6-31G* computational level using QCforever (https://github.com/molecule-generator-collection/QCforever) or similar tools (for computational details, refer to paper XXX).
- First, perform the following Setup once at the beginning, and then carry out Visualization (1 or 2).
- In Visualizations 1 and 2, a sample property list and property values are already provided for checking the visualization. After executing the cell, press the Run button.
- Enter the property list and the property values of the molecule you have computed and press the Run button. MolAtlas will visualize the molecule in the property space.


In [25]:
#@title Setup (Please execute this only once at the beginning. It may take several minutes.)

%%capture
!pip install pycirclize

import os
import csv
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.colors as mcolors
from pycirclize import Circos
import matplotlib.pyplot as plt
from scipy.interpolate import interpn, griddata
from matplotlib.colors import ListedColormap


#Road percentile data
!wget --no-check-certificate 'https://drive.google.com/uc?id=1TLJ-Ixjw43QhfbgyklNaGsVIiextts26' -O DBZINC_charge0.csv
!wget --no-check-certificate 'https://drive.google.com/uc?id=1k1wk9CdPBkrWXOSIg_bw1pQkTkQxIQpX' -O DBZINC_chargeAll.csv

!wget --no-check-certificate 'https://drive.google.com/uc?id=1UufyVf-NkBzT1ulKD36PFEes2MqdresE' -O DBPubChem_charge0.csv
!wget --no-check-certificate 'https://drive.google.com/uc?id=1KUaSkCvnEToOw6-5nX5Xknn-8K3Eqa9c' -O DBPubChem_chargeAll.csv

!wget --no-check-certificate 'https://drive.google.com/uc?id=1-7KkQAvMuFRb4z3yRrpLWhnI_h5KTYW4' -O DBGDB_charge0.csv
!wget --no-check-certificate 'https://drive.google.com/uc?id=1KWcLTFDL3eem9SHWL835U0Vo2Fe7Mp3a' -O DBGDB_chargeAll.csv

!wget --no-check-certificate 'https://drive.google.com/uc?id=1E-XcIzi21_RI34G8czimu8BlStLSDyqm' -O DBAll_charge0.csv
!wget --no-check-certificate 'https://drive.google.com/uc?id=1o0_ID4fqSIJ__3RKNKZ6kdoX6rkSnX1f' -O DBAll_chargeAll.csv


DBZINC_charge0_df = pd.read_csv('DBZINC_charge0.csv', skiprows=[1])  # Skip the second row
DBZINC_chargeAll_df = pd.read_csv('DBZINC_chargeAll.csv', skiprows=[1])  # Skip the second row

DBPubChem_charge0_df = pd.read_csv('DBPubChem_charge0.csv', skiprows=[1])  # Skip the second row
DBPubChem_chargeAll_df = pd.read_csv('DBPubChem_chargeAll.csv', skiprows=[1])  # Skip the second row

DBGDB_charge0_df = pd.read_csv('DBGDB_charge0.csv', skiprows=[1])  # Skip the second row
DBGDB_chargeAll_df = pd.read_csv('DBGDB_chargeAll.csv', skiprows=[1])  # Skip the second row

DBAll_charge0_df = pd.read_csv('DBAll_charge0.csv', skiprows=[1])  # Skip the second row
DBAll_chargeAll_df = pd.read_csv('DBAll_chargeAll.csv', skiprows=[1])  # Skip the second row

#Road kde data
!wget --no-check-certificate 'https://drive.google.com/uc?id=1lXAtkPYZRY5RyP7rrXNSj58FDTq_hHTS' -O kde_DBZINC_charge0.pickle
!wget --no-check-certificate 'https://drive.google.com/uc?id=1Gjh-AXN2jwG1_vBS4X6ZZm4dgVfdbNx1' -O kde_DBZINC_chargeAll.pickle

!wget --no-check-certificate 'https://drive.google.com/uc?id=1Xi34k8OlxBmodxr5JHWTiLhOXThXw1Ah' -O kde_DBPubChem_charge0.pickle
!wget --no-check-certificate 'https://drive.google.com/uc?id=1pfv6b-h98LqpTRNA7OzWUCTwFsaFEq-P' -O kde_DBPubChem_chargeAll.pickle

!wget --no-check-certificate 'https://drive.google.com/uc?id=1bkxcukXK-pkm8xnAP5C1gauBkK3Bmo-r' -O kde_DBGDB_charge0.pickle
!wget --no-check-certificate 'https://drive.google.com/uc?id=1fb84P4KVjv6L5_bWDJR8ssr9bMRhA9nu' -O kde_DBGDB_chargeAll.pickle

!wget --no-check-certificate 'https://drive.google.com/uc?id=1qlc3NSkgJCgwKNzq3LUcLgNWphrg0RY8' -O kde_DBAll_charge0.pickle
!wget --no-check-certificate 'https://drive.google.com/uc?id=1HuV4nDFFCvkM0bh1mZtwnSV8HfzSTP0_' -O kde_DBAll_chargeAll.pickle

with open(f'kde_DBZINC_charge0.pickle', mode='rb') as f:
    kde_dict_DBZINC_charge0 = pickle.load(f)
with open(f'kde_DBZINC_chargeAll.pickle', mode='rb') as f:
    kde_dict_DBZINC_chargeAll = pickle.load(f)

with open(f'kde_DBPubChem_charge0.pickle', mode='rb') as f:
    kde_dict_DBPubChem_charge0 = pickle.load(f)
with open(f'kde_DBPubChem_chargeAll.pickle', mode='rb') as f:
    kde_dict_DBPubChem_chargeAll = pickle.load(f)

with open(f'kde_DBGDB_charge0.pickle', mode='rb') as f:
    kde_dict_DBGDB_charge0 = pickle.load(f)
with open(f'kde_DBGDB_chargeAll.pickle', mode='rb') as f:
    kde_dict_DBGDB_chargeAll = pickle.load(f)

with open(f'kde_DBAll_charge0.pickle', mode='rb') as f:
    kde_dict_DBAll_charge0 = pickle.load(f)
with open(f'kde_DBAll_chargeAll.pickle', mode='rb') as f:
    kde_dict_DBAll_chargeAll = pickle.load(f)



# Function to execute when button is clicked (percentile)
def on_execute_button_clicked(b):
    with output_area_1:
        clear_output(wait=True)  # Clear previous output

        # Get user inputs
        selected_database = database_dropdown_1.value
        selected_properties = [prop.strip() for prop in property_textbox_1.value.split(",") if prop.strip()]
        selected_data = [float(val.strip()) for val in data_textbox_1.value.split(",") if val.strip()]
        selected_charge = charge_dropdown_1.value

        # Select appropriate dataframe based on user choice
        if selected_charge == "0":
            if selected_database == "ZINC":
                data_df = DBZINC_charge0_df
            elif selected_database == "PubChem":
                data_df = DBPubChem_charge0_df
            elif selected_database == "GDB":
                data_df = DBGDB_charge0_df
            elif selected_database == "All":
                data_df = DBAll_charge0_df
        elif selected_charge == "All":
            if selected_database == "ZINC":
                data_df = DBZINC_chargeAll_df
            elif selected_database == "PubChem":
                data_df = DBPubChem_chargeAll_df
            elif selected_database == "GDB":
                data_df = DBGDB_chargeAll_df
            elif selected_database == "All":
                data_df = DBAll_chargeAll_df

        # Percentile and Z-score calculation
        def calc_percentile_lookup(new_data, percentile_values):
            percentiles = np.linspace(0, 100, len(percentile_values))
            percentile_lookup = np.interp(new_data, percentile_values, percentiles)
            return percentile_lookup

        percentile_list = []
        zscore_list = []

        for i, prop in enumerate(selected_properties):
            tmp_data = data_df[prop].values
            percentile = calc_percentile_lookup(selected_data[i], tmp_data[2:])
            percentile_list.append(float(percentile))

            mean, SD = tmp_data[0], tmp_data[1]
            z_score = (selected_data[i] - mean) / SD  # Z-score calculation
            zscore_list.append(float(z_score))

        # Display calculated values
        print("Execution Complete ✅\n")
        print(f"📂 Selected Database: {selected_database}")
        print(f"🔬 Selected Properties: {selected_properties}")
        print(f"📊 Input Data: {selected_data}")
        print(f"⚡ Charge Condition: {selected_charge}")
        print(f"📈 Percentile List: {percentile_list}")
        print(f"📉 Z-score List: {zscore_list}")

        # Create Radar Chart
        df = pd.DataFrame(
            data=[
                percentile_list,
                [50] * len(selected_properties),
            ],
            index=["Target mol.", "Median"],
            columns=[
                #f"{selected_properties[i]}\n {percentile_list[i]:.2f}th, Z={zscore_list[i]:.2f}"
                f"{selected_properties[i]}\n{selected_data[i]}\n({percentile_list[i]:.2f}th)"
                for i in range(len(selected_properties))
            ],
        )

        circos = Circos.radar_chart(
            df,
            vmax=100,
            marker_size=6,
            grid_interval_ratio=0.2,
            grid_label_formatter=lambda v: f"{v:.0f} th",
            label_kws_handler=lambda _: dict(size=16),
        )

        fig = circos.plotfig()
        _ = circos.ax.legend(bbox_to_anchor=(0.97, 1.15), loc='upper left', fontsize=16)

        os.makedirs(f'fig', exist_ok=True)
        plt.savefig(f'fig/radar_DB{selected_database}_charge{selected_charge}.pdf',
                transparent=True, dpi = 300)
        print(f'💻 Saved radar chart: fig/radar_DB{selected_database}_charge{selected_charge}.pdf')

        plt.show()

        # Create Violin Plots
        fig, axs = plt.subplots(nrows=1, ncols=len(selected_properties), figsize=(len(selected_properties) * 2, 5))

        for i, prop in enumerate(selected_properties):
            tmp_data = data_df[prop].values
            interp_indices = np.linspace(0, 100, len(tmp_data[2:]))
            reconstructed_data = np.interp(interp_indices, interp_indices, tmp_data[2:])

            axs[i].violinplot(reconstructed_data, showmeans=False, showmedians=True)
            axs[i].set_xticks([1], labels=[prop], fontsize=14)
            axs[i].scatter([[1]], [[selected_data[i]]],s=40, marker = "o", c="black")
            axs[i].set_title(f"{selected_data[i]} ({percentile_list[i]:.2f}th)\n(Z-score={zscore_list[i]:.2f})", fontsize=14)
            axs[i].tick_params(axis='both', which='major', labelsize=12)
            axs[i].tick_params(axis='both', which='minor', labelsize=12)

        plt.tight_layout()
        plt.savefig(f'fig/1D_dist_DB{selected_database}_charge{selected_charge}.pdf',
                transparent=True, dpi = 300)
        print(f'💻 Saved plot: fig/1D_dist_DB{selected_database}_charge{selected_charge}.pdf')

        plt.show()

def compute_density_and_percentile(given_data, kde_info, prop_pair, selected_database, selected_charge):
    """
    Computes the density and percentile of a given data point based on the precomputed KDE grid.
    """

    X_info, Y_info, Z, scaler_x, scaler_y, contour_levels, percentile_levels = kde_info
    X, Y = np.meshgrid(np.linspace(X_info[0], X_info[1], X_info[2]), np.linspace(Y_info[0], Y_info[1], Y_info[2]))

    # データをスケーリング
    if len(given_data) == 2:
      scaled_x = scaler_x.transform([[given_data[0]]])[0, 0]
      scaled_y = scaler_y.transform([[given_data[1]]])[0, 0]
    X_inv = scaler_x.inverse_transform(X)
    Y_inv = scaler_y.inverse_transform(Y)

    # KDE 密度推定（補間）
    if len(given_data) == 2:
      density_a = griddata(
          points=np.column_stack([X.ravel(), Y.ravel()]),
          values=Z.ravel(),
          xi=np.array([[scaled_x, scaled_y]]),
          method="linear"
      )[0]

    # パーセンタイル計算
    Z_sorted = np.sort(Z.ravel())
    cumsum = np.cumsum(Z_sorted) / np.sum(Z_sorted)
    if len(given_data) == 2:
      percentile_a = (cumsum[np.argmax(Z_sorted >= density_a)] * 100)

    #帯の描画用
    percentilef_levels = [0.0001,0.0002,0.0005, 0.001,0.002,0.005,
                          0.01,0.02,0.05, 0.1, 0.2, 0.5]# [0.0001, 0.0002, 0.0005, 0.001,0.01, 0.1, 0.2, 0.5, 1],
    contourf_levels = [Z_sorted[np.argmax(cumsum >= p)] for p in percentilef_levels]
    contourf_levels = sorted(contourf_levels)


    # `levels` の設定（最小値をZ_minから開始）
    levels = [0]+contourf_levels+[np.max(Z)]

    # カラーマップの設定（最小値未満の部分は白）
    cmap_original = sns.cubehelix_palette(light=1, as_cmap=True)  # オリジナルのカラーマップ
    new_colors = cmap_original(np.linspace(0, 1, len(levels)))  # `levels` の数に合わせる
    new_colors[0] = [1, 1, 1, 1]  # 最小値未満の領域を白にする
    custom_cmap = mcolors.ListedColormap(new_colors)

    # `BoundaryNorm` を使用して `levels` を適切に適用
    norm = mcolors.BoundaryNorm(levels, custom_cmap.N)

    # `contourf` を描画
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_box_aspect(1)
    contourf = ax.contourf(X_inv, Y_inv, Z, levels=levels, cmap=custom_cmap, norm=norm)

    # カラーバーを追加
    cbar = plt.colorbar(contourf, ax=ax, shrink=0.8)
    cbar.set_label("Estimated density", size=10)
    cbar.set_ticks(levels[1:])  # Set tick positions at levels (excluding 0)
    cbar.set_ticklabels([f"{tick:.1e}" for tick in levels[1:]], fontsize=10)  # Convert to scientific notation



    contour = ax.contour(X_inv, Y_inv, Z, levels=contour_levels, colors=['indigo', 'darkmagenta', 'mediumvioletred', 'white'])
    ax.clabel(contour, fmt={l: f"{p*100:.2f}%" for l, p in zip(contour_levels, percentile_levels)}, fontsize=8)

    if len(given_data) == 2:
        plt.scatter([given_data[0]], [given_data[1]],
                    c = 'orange', s = 40, marker = "X", label = f'Computed data\n(percentile of density = {percentile_a:.2f}%)')
        plt.legend(loc='upper right', framealpha=0, edgecolor="#ff0000", fontsize = 12)
    plt.xlabel(prop_pair[0], fontsize = 16)
    plt.ylabel(prop_pair[1], fontsize = 16)

    os.makedirs(f'fig', exist_ok=True)
    saved_file_path = f'fig/2D_dist_DB{selected_database}_charge{selected_charge}_pr1{prop_pair[0]}_pr2{prop_pair[1]}.pdf'
    plt.savefig(saved_file_path, transparent=True, dpi = 300)


    plt.show()

    if len(given_data) == 2:
      return density_a, percentile_a, saved_file_path
    else:
      return None, None, saved_file_path


# Function to execute when button is clicked (Kernel Density Estimation; KDE)
def on_execute_button_clicked_kde(b):
    with output_area_2:
        clear_output(wait=True)  # Clear previous output

        # Get user inputs
        selected_database = database_dropdown_2.value
        selected_properties = [prop.strip() for prop in property_textbox_2.value.split(",") if prop.strip()]
        selected_data = [float(val.strip()) for val in data_textbox_2.value.split(",") if val.strip()]
        selected_charge = charge_dropdown_2.value

        # Select appropriate dataframe based on user choice
        if selected_charge == "0":
            if selected_database == "ZINC":
                kde_dict = kde_dict_DBZINC_charge0
            elif selected_database == "PubChem":
                kde_dict = kde_dict_DBPubChem_charge0
            elif selected_database == "GDB":
                kde_dict = kde_dict_DBGDB_charge0
            elif selected_database == "All":
                kde_dict = kde_dict_DBAll_charge0
        elif selected_charge == "All":
            if selected_database == "ZINC":
                kde_dict = kde_dict_DBZINC_chargeAll
            elif selected_database == "PubChem":
                kde_dict = kde_dict_DBPubChem_chargeAll
            elif selected_database == "GDB":
                kde_dict = kde_dict_DBGDB_chargeAll
            elif selected_database == "All":
                kde_dict = kde_dict_DBAll_chargeAll

        kde_info = kde_dict[f'DB{selected_database}_charge{selected_charge}_p1{selected_properties[0]}_p2{selected_properties[1]}']

        density, percentile, saved_file_path = compute_density_and_percentile(selected_data, kde_info, selected_properties, selected_database, selected_charge)

        # Display calculated values
        print("Execution Complete ✅\n")
        print(f"📂 Selected Database: {selected_database}")
        print(f"🔬 Selected Properties: {selected_properties}")
        print(f"📊 Input Data: {selected_data}")
        print(f"⚡ Charge Condition: {selected_charge}")
        print(f"📈 Density: {density}")
        print(f"📉 Persentile of density: {percentile}")
        print(f"💻 Saved plot: {saved_file_path}")



In [23]:
#@title 1. Visualizing where your calculated molecule is located in each property

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycirclize import Circos


# Database selection
database_label_1 = widgets.Label(value="Select the database:")
database_options_1 = ["All", "ZINC", "PubChem", "GDB"]
database_dropdown_1 = widgets.Dropdown(
    options=database_options_1,
    value="All",
    description="DB:",
    disabled=False
)

# List of available properties
property_list = [
    "MW", "HOMO", "LUMO", "HOMO-LUMO gap", "VIP", "VEA", "Dipole", "Energy", "DEEN", "Chrg_var",
    "Spin_sum", "Freq", "E_free", "E_enth", "Ei", "Et", "Ezp", "Cv", "Si", "polar_iso", "polar_aniso",
    "IR", "Raman", "Abs_wl_1", "Abs_f_1", "CD_mu_1", "CD_theta_1", "CD_g_1",
    "FL_wl_1", "FL_f_1", "CPL_mu_1", "CPL_theta_1", "CPL_g_1", "charge"
]

# Display available properties correctly
property_list_html_1 = widgets.HTML("<b>Available properties:</b><br>" + ", ".join(property_list))

# Default property list
default_properties_1 = "MW, HOMO, LUMO, HOMO-LUMO gap, Abs_wl_1, Abs_f_1"
default_values_1 = "300, -6, -2, 4, 350, 0.03"

# Property list input
property_label_1 = widgets.Label(value="Enter the list of properties (comma-separated):")
property_textbox_1 = widgets.Textarea(
    value=default_properties_1,
    layout=widgets.Layout(width="90%", height="50px")
)

# Calculation results input
data_label_1 = widgets.Label(value="Enter the corresponding calculation results (comma-separated):")
data_textbox_1 = widgets.Textarea(
    value=default_values_1,
    layout=widgets.Layout(width="90%", height="50px")
)

# Charge condition selection
charge_label_1 = widgets.Label(value="Select charge condition (0: neutral molecules only, All: all molecules):")
charge_dropdown_1 = widgets.Dropdown(
    options=["0", "All"],
    value="0",
    description="Charge:",
    disabled=False
)

# Output area for results
output_area_1 = widgets.Output()

# Execute button
execute_button_1 = widgets.Button(description="Run", button_style="primary")
execute_button_1.on_click(on_execute_button_clicked)

# Display the UI
#ui_1 = widgets.VBox([database_label_1, database_dropdown_1,
#                      property_label_1, property_textbox_1, property_list_html_1,
#                      data_label_1, data_textbox_1, charge_label_1, charge_dropdown_1,
#                      execute_button_1, output_area_1])
#display(ui_1)

display(database_label_1, database_dropdown_1)
display(property_label_1, property_textbox_1)
display(property_list_html_1)
display(data_label_1, data_textbox_1)
display(charge_label_1, charge_dropdown_1)
display(execute_button_1)
display(output_area_1)


Label(value='Select the database:')

Dropdown(description='DB:', options=('All', 'ZINC', 'PubChem', 'GDB'), value='All')

Label(value='Enter the list of properties (comma-separated):')

Textarea(value='MW, HOMO, LUMO, HOMO-LUMO gap, Abs_wl_1, Abs_f_1', layout=Layout(height='50px', width='90%'))

HTML(value='<b>Available properties:</b><br>MW, HOMO, LUMO, HOMO-LUMO gap, VIP, VEA, Dipole, Energy, DEEN, Chr…

...

Textarea(value='300, -6, -2, 4, 350, 0.03', layout=Layout(height='50px', width='90%'))

Label(value='Select charge condition (0: neutral molecules only, All: all molecules):')

Dropdown(description='Charge:', options=('0', 'All'), value='0')

Button(button_style='primary', description='Run', style=ButtonStyle())

Output()

In [24]:
#@title 2. Visualizing where your calculated molecule is located in 2D space defined by two properties.


# Database selection
database_label_2 = widgets.Label(value="Select the database:")
database_options_2 = ["All", "ZINC", "PubChem", "GDB"]
database_dropdown_2 = widgets.Dropdown(
    options=database_options_2,
    value="All",
    description="DB:",
    disabled=False
)

# List of available properties
property_list = [
    "MW", "HOMO", "LUMO", "HOMO-LUMO gap", "VIP", "VEA", "Dipole", "Energy", "DEEN", "Chrg_var",
    "Spin_sum", "Freq", "E_free", "E_enth", "Ei", "Et", "Ezp", "Cv", "Si", "polar_iso", "polar_aniso",
    "IR", "Raman", "Abs_wl_1", "Abs_f_1", "CD_mu_1", "CD_theta_1", "CD_g_1",
    "FL_wl_1", "FL_f_1", "CPL_mu_1", "CPL_theta_1", "CPL_g_1", "charge"
]

# Display available properties correctly
property_list_html_2 = HTML("<b>Available properties:</b><br>" + ", ".join(property_list))

# Default property list
default_properties_2 = "MW, HOMO"
default_values_2 = "300, -6"

# Property list input
property_label_2 = widgets.Label(value="Enter a pair of properties (comma-separated):")
property_textbox_2 = widgets.Textarea(
    value=default_properties_2,
    layout=widgets.Layout(width="90%", height="50px")
)

# Calculation results input
data_label_2 = widgets.Label(value="Enter the corresponding calculation results (comma-separated):")
data_textbox_2 = widgets.Textarea(
    value=default_values_2,
    layout=widgets.Layout(width="90%", height="50px")
)

# Charge condition selection
charge_label_2 = widgets.Label(value="Select charge condition (0: neutral molecules only, All: all molecules):")
charge_dropdown_2 = widgets.Dropdown(
    options=["0", "All"],
    value="0",
    description="Charge:",
    disabled=False
)

# Output area for results
output_area_2 = widgets.Output()

# Execute button
execute_button_2 = widgets.Button(description="Run", button_style="primary")
execute_button_2.on_click(on_execute_button_clicked_kde)

# Display the UI

display(database_label_2, database_dropdown_2)
display(property_label_2, property_textbox_2)
display(property_list_html_2)
display(data_label_2, data_textbox_2)
display(charge_label_2, charge_dropdown_2)
display(execute_button_2)
display(output_area_2)


Label(value='Select the database:')

Dropdown(description='DB:', options=('All', 'ZINC', 'PubChem', 'GDB'), value='All')

Label(value='Enter a pair of properties (comma-separated):')

Textarea(value='MW, HOMO', layout=Layout(height='50px', width='90%'))

Label(value='Enter the corresponding calculation results (comma-separated):')

Textarea(value='300, -6', layout=Layout(height='50px', width='90%'))

Label(value='Select charge condition (0: neutral molecules only, All: all molecules):')

Dropdown(description='Charge:', options=('0', 'All'), value='0')

Button(button_style='primary', description='Run', style=ButtonStyle())

Output()